In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt install libomp-dev
!pip install faiss
!pip install faiss-gpu
!pip install annoy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

from sklearn.neighbors import NearestNeighbors

from PIL import Image
import math
import numpy as np
from sklearn.utils import shuffle
import os
import json
from sklearn.model_selection import train_test_split
import cv2
from google.colab.patches import cv2_imshow
from scipy import spatial

import re

In [ ]:
import matplotlib.pyplot as plt

def draw_image(path):
    img = cv2.imread(path)
    cv2_imshow(img)
def draw_list_image(paths, titles = None):
    axes = []
    grid_size = 5
    k = len(paths)
    fig = plt.figure(figsize=(20, 20))
    for id in range(k):
        path = "/content/drive/MyDrive/Yonin-IR/data/paris_data/" + paths[id].split('/')[-1] + '.jpg'
        print(path)
        axes.append(fig.add_subplot(grid_size, grid_size, id + 1))
        if (titles != None):
            axes[-1].set_title(round(titles[id], 5))
        plt.imshow(Image.open(path))
    fig.tight_layout()
    plt.show()

##Load Data

In [ ]:
data_path = "/content/drive/MyDrive/data/oxbuild_index.json"
with open(data_path) as fp:
    data = json.load(fp)

In [ ]:
vectors = np.array(data['vectors'])
paths = data['paths']
names = data['names']

In [ ]:
vectors = vectors.astype(np.float32)

In [ ]:
vectors.shape

(5063, 512)

##Faiss

In [ ]:
import faiss
flower_index = faiss.IndexFlatL2(vectors.shape[1])
flower_index.add(vectors)

##Load query

In [ ]:
def extract_file_names(path):
    listdir = os.listdir(path)
    list_name = []
    for file in listdir:
        type_file = file.split('_')[-1][:-4]
        if type_file == 'good':
            list_name.append(file[:-9])
    return list_name

In [ ]:
query_file_path = "/content/drive/MyDrive/data/querry_oxfordbuild"
query_file = extract_file_names(query_file_path)

In [ ]:
len(query_file)

55

In [ ]:
query_file[:5]

['all_souls_4', 'all_souls_1', 'all_souls_3', 'all_souls_2', 'balliol_5']

In [ ]:
path = "/content/drive/MyDrive/data/oxbuild_index.json"
with open(path) as fp:
    query = json.load(fp)

In [ ]:
query['names'][:5]

['oxford_001802',
 'magdalen_000316',
 'jesus_000336',
 'jesus_000117',
 'christ_church_000802']

In [ ]:
def remove_trailing_zeros(input_string):
    parts = input_string.rsplit('_', 1)

    if len(parts) == 2:
        prefix, number_part = parts
        modified_number = re.sub(r'^0+', '', number_part)
        modified_string = f'{prefix}_{modified_number}'
        return modified_string

    return input_string

In [ ]:
vectors_query = {}
for i in range(len(query['names'])):
    original_string = query['names'][i]
    modified_string = remove_trailing_zeros(original_string)
    print(modified_string)
    vectors_query[modified_string] = query['vectors'][i]


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
christ_church_736
oxford_1675
balliol_45
magdalen_477
oxford_2575
christ_church_658
magdalen_751
new_98
oxford_809
oxford_400
oxford_1992
oxford_1307
christ_church_283
oxford_530
magdalen_39
oxford_2569
magdalen_647
magdalen_179
oxford_1522
trinity_196
ashmolean_257
christ_church_182
jesus_99
magdalen_247
oxford_609
ashmolean_250
ashmolean_14
oxford_3073
magdalen_32
oxford_2670
magdalen_1142
radcliffe_camera_426
christ_church_449
trinity_130
cornmarket_131
oxford_1239
balliol_156
oxford_1048
jesus_307
magdalen_529
radcliffe_camera_25
hertford_34
all_souls_14
trinity_309
oriel_87
oxford_3261
ashmolean_169
ashmolean_145
christ_church_767
radcliffe_camera_509
magdalen_965
new_790
trinity_113
oxford_967
all_souls_220
trinity_2
oxford_7
radcliffe_camera_170
bodleian_294
magdalen_151
christ_church_1028
new_404
oxford_2049
oxford_2713
magdalen_452
oxford_2401
oxford_37
magdalen_373
magdalen_141
hertford_8
radcliffe_camera_386
christ_chu

In [ ]:
vectors_query['all_souls_1']

[0.014550743624567986,
 0.03476293757557869,
 0.010319093242287636,
 0.057365138083696365,
 6.96926110571594e-12,
 6.96926110571594e-12,
 0.002713096560910344,
 0.011018446646630764,
 6.96926110571594e-12,
 6.96926110571594e-12,
 6.96926110571594e-12,
 6.96926110571594e-12,
 0.048425957560539246,
 6.96926110571594e-12,
 6.96926110571594e-12,
 0.03033909574151039,
 6.96926110571594e-12,
 0.12480785697698593,
 0.04471724480390549,
 0.04545425623655319,
 0.0828084722161293,
 0.015966063365340233,
 6.96926110571594e-12,
 6.96926110571594e-12,
 6.96926110571594e-12,
 0.015351228415966034,
 0.02246881276369095,
 6.96926110571594e-12,
 0.043581392616033554,
 0.007316398900002241,
 0.015163982287049294,
 6.96926110571594e-12,
 6.96926110571594e-12,
 6.96926110571594e-12,
 0.0836583599448204,
 0.0076855323277413845,
 6.96926110571594e-12,
 0.032196044921875,
 6.96926110571594e-12,
 0.010705292224884033,
 6.96926110571594e-12,
 6.96926110571594e-12,
 0.07203452289104462,
 0.010044806636869907,
 

##List rank for each query

In [ ]:
def search1(index, names, query_vector, k):
    distances, indices = index.search(np.expand_dims(query_vector, axis=0), k)

    # Extract the names of the nearest neighbors
    nearest_paths = [names[i] for i in indices[0]]

    return nearest_paths, distances

In [ ]:
import faiss
import numpy as np
from scipy import spatial


vectors = np.array(data['vectors'])
def re_ranking(initial_results, distance):

    combined_results = list(zip(initial_results, distance, range(len(initial_results))))

    # Sort the combined results by the refined distance metric (you can customize this metric)
    combined_results.sort(key=lambda x: (x[1], -x[2]))

    # Extract the re-ranked image IDs
    re_ranked_results = [result[0] for result in combined_results]

    return re_ranked_results

def search(index, names, query_vector, k, data_vectors = vectors):
    # Convert the query vector to float32 if needed
    distances = [spatial.distance.cosine(i, query_vector) for i in data_vectors]
    ids = np.argsort(distances)[:k]
    results = [data['paths'][id] for id in ids]
    results = re_ranking(results,distances)

    return results, distances


In [ ]:
from annoy import AnnoyIndex
import numpy as np

def build_annoy_index(vectors = data['vectors'], metric='angular'):
    # Assuming vectors is a list of vectors
    dimension = len(vectors[0])
    index = AnnoyIndex(dimension, metric=metric)

    for i, vector in enumerate(vectors):
        index.add_item(i, vector)

    index.build(n_trees=10)  # You can adjust the number of trees for performance

    return index

def search_annoy(index, query_vector, k):
    # Perform a search
    indices = index.get_nns_by_vector(query_vector, k)

    return indices

# Example usage:
# your_vectors_list = [...]  # Replace with your actual vectors
# index = build_annoy_index(your_vectors_list)
# query_vector = np.random.rand(512)  # Replace with your actual query vector
# result_indices = search_annoy(index, query_vector, k=10)
# result_paths = [your_names_list[i] for i in result_indices]

In [ ]:
image_path = "/content/drive/MyDrive/data/data_oxbuild/"
for name in query_file:
    if name in vectors_query:
        query_vector = np.array(vectors_query[name])
        query_vector = query_vector.astype(np.float32)
        nearest_paths, nearest_distances = search(flower_index, names, query_vector, 5000)
        list_rank_path = "/content/drive/MyDrive/data/oxford_list_rank/" + name + ".json"
        with open(list_rank_path, "w") as fp:
            json.dump(nearest_paths, fp, indent=4)
    else:
        print(f"Vector for name {name} not found. Skipping...")



Vector for name all_souls_4 not found. Skipping...
Vector for name balliol_5 not found. Skipping...
Vector for name balliol_2 not found. Skipping...
Vector for name ashmolean_1 not found. Skipping...
Vector for name bodleian_3 not found. Skipping...
Vector for name balliol_1 not found. Skipping...
Vector for name ashmolean_4 not found. Skipping...
Vector for name balliol_3 not found. Skipping...
Vector for name balliol_4 not found. Skipping...
Vector for name cornmarket_1 not found. Skipping...
Vector for name cornmarket_5 not found. Skipping...
Vector for name cornmarket_3 not found. Skipping...
Vector for name bodleian_5 not found. Skipping...
Vector for name christ_church_4 not found. Skipping...
Vector for name cornmarket_2 not found. Skipping...
Vector for name bodleian_4 not found. Skipping...
Vector for name hertford_1 not found. Skipping...
Vector for name magdalen_3 not found. Skipping...
Vector for name hertford_2 not found. Skipping...
Vector for name hertford_4 not found. S

In [ ]:
# # from sklearn.neighbors import NearestNeighbors
# # import json
# # import numpy as np

# # Assuming vectors_query is a dictionary containing vector representations for each image name

# # Extract vectors and corresponding names


# # Create a NearestNeighbors model with Euclidean distance (default)
# # neighbors_model = NearestNeighbors(n_neighbors=5000, algorithm='auto', metric='euclidean')

# # Fit the model with your vectors
# # neighbors_model.fit(vectors)

# # Perform search for each query
# # for name in query_file:
# #     if name in vectors_query:
# #         query_vector = np.array(vectors_query[name]).reshape(1, -1)  # Reshape to match the input format
# #         distances, indices = neighbors_model.kneighbors(query_vector)

# #         Get the nearest paths
# #         nearest_paths = [names[index] for index in indices.flatten()]

# #         Save the results to a JSON file
# #         list_rank_path = "/content/drive/MyDrive/data/oxford_list_rank/" + name + ".json"
# #         with open(list_rank_path, "w") as fp:
# #             json.dump(nearest_paths, fp, indent=4)
# #     else:
# #         print(f"Vector for name {name} not found. Skipping...")

##Evaluation

###MAP

In [ ]:
k = 2000

list_rank_path = "/content/drive/MyDrive/data/oxford_list_rank/"
query_file_path = "/content/drive/MyDrive/data/querry_oxfordbuild/"
list_ap = []
for name in query_file:
    try:
        with open(list_rank_path + name + ".json") as fp:
            list_rank = json.load(fp)
    except FileNotFoundError:
        print(f"File not found: {list_rank_path + name + '.json'}. Ignoring...")
        continue
    with open(query_file_path + name + "_good.txt", 'r') as fp:
        good_set = fp.read().split('\n')[:-1]
    with open(query_file_path + name + "_ok.txt", 'r') as fp:
        ok_set = fp.read().split('\n')[:-1]
    with open(query_file_path + name + "_junk.txt", 'r') as fp:
        junk_set = fp.read().split('\n')[:-1]
    right_set = good_set + ok_set

    pos = len(right_set)
    ap = 0
    relevant = 0
    old_recall = 0
    old_pre = 1
    j = 0
    for i in range(k):
        if any(s in list_rank[i] for s in right_set):
            relevant += 1
        elif any(s in list_rank[i] for s in junk_set):
            continue
        pre = relevant / (j + 1)
        recall = relevant/pos
        ap += (recall - old_recall)*((pre + old_pre)/2)
        j += 1
        old_recall = recall
        old_pre = pre
    list_ap.append(np.mean(ap))
map = np.mean(list_ap)
print(map)

File not found: /content/drive/MyDrive/data/oxford_list_rank/all_souls_4.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/balliol_5.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/balliol_2.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/ashmolean_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/bodleian_3.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/balliol_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/ashmolean_4.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/balliol_3.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/balliol_4.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/cornmarket_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/cornmarket_5.json. Ignoring...
File not

###MAP@K

In [ ]:
k = 5
list_rank_path = "/content/drive/MyDrive/data/oxford_list_rank/"
query_file_path = "/content/drive/MyDrive/data/querry_oxfordbuild/"
ap = []
for name in query_file:
    try:
        with open(list_rank_path + name + ".json") as fp:
            list_rank = json.load(fp)
    except FileNotFoundError:
        print(f"File not found: {list_rank_path + name + '.json'}. Ignoring...")
        continue
    with open(query_file_path + name + "_good.txt", 'r') as fp:
        good_set = fp.read().split('\n')[:-1]
    with open(query_file_path + name + "_ok.txt", 'r') as fp:
        ok_set = fp.read().split('\n')[:-1]
    with open(query_file_path + name + "_junk.txt", 'r') as fp:
        junk_set = fp.read().split('\n')[:-1]
    right_set = good_set + ok_set

    precision = []
    relevant = 0
    j = 0
    for i in range(k):
        if any(s in list_rank[i] for s in right_set):
            relevant += 1
        elif any(s in list_rank[i] for s in junk_set):
            continue
        pre = relevant / (j + 1)
        precision.append(pre)
        j += 1
    ap.append(np.mean(precision))
map = np.mean(ap)
print('top ' + str(k) + ': ', map)

File not found: /content/drive/MyDrive/data/oxford_list_rank/all_souls_4.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/balliol_5.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/balliol_2.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/ashmolean_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/bodleian_3.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/balliol_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/ashmolean_4.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/balliol_3.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/balliol_4.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/cornmarket_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/oxford_list_rank/cornmarket_5.json. Ignoring...
File not

In [ ]:
ap

[0.009882004232837428,
 0.0,
 0.0,
 0.0,
 0.007619906957679846,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.00566569621444406,
 0.06728802361210524,
 0.0,
 0.012154971701546856,
 0.0,
 0.0,
 0.0,
 0.0,
 0.03514917689238287,
 0.0,
 0.0025295889000065026,
 0.0,
 0.0,
 0.0]

##Visualize Result